히스토그램 & 밝기 조절

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tkinter import *

In [4]:
path = 'C:/Users/CYM/Desktop/DIP_image'
src_path = path+'/src/'
label_path = path+'/label/'

sample_image1 = 'C:/Users/CYM/Desktop/DIP_image/src/18-40-02-02 (SLA).jpg'
sample_image2 = 'C:/Users/CYM/Desktop/DIP_image/src/clahe.png'


In [11]:
'''
폴더에서 n번째 이미지를 가져올 수 있는 옵션을 사용자에게 줄까요?
다만 이거까지 주게되면 총 3개의 입력값을 받아야 해서 지져분해 보일것 같긴 해요.

'''
class A:    
    def __init__(self):
        self.window = Tk()
        self.window.geometry('500x500')
        aa=Label(self.window,text='주의! 미리보기 종료시, 반드시 아무키나 눌러주세요!').pack()
        aa=Label(self.window,text='ㅇㅇㅇㅇㅇㅇㅇㅇㅇ').pack()
        self.c = Entry(self.window)
        self.b = Entry(self.window)
        self.c.insert(0,"0")
        self.b.insert(0,"0")
        self.c.pack()
        self.b.pack()
        
        cl_bm_but = Button(self.window, text="미리보기",command=self.getText)
        cl_bm_but.pack()    # CLAHE_BM 실행 버튼

        finish_but = Button(self.window, text="실행/Tk종료", command=self.fin)
        finish_but.pack()

        self.window.mainloop()
    
    # 버튼으로 tkinter 종료
    def fin(self):
        self.window.destroy()
        
    # Entry값 가져오기
    def getText(self):
        k = float(self.c.get())
        w = int(self.b.get())
        self.CLAHE_BN(k,w)



    def CLAHE_BN(self, clip, br):
        img_list = os.listdir(src_path)  
        #src폴더 내에 n번째 이미지 불러오기
        set_img_n = cv2.imread(src_path + img_list[0])                  

        sample_image = cv2.cvtColor(set_img_n, cv2.COLOR_BGR2YUV)
        clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=(8,8))

        # CLAHE setting
        if clip != 0:
            sample_image[:,:,0] = clahe.apply(sample_image[:,:,0])

        # brightness setting
        sample_image[:,:,0] = cv2.add(sample_image[:,:,0], br)


        sample_image = cv2.cvtColor(sample_image, cv2.COLOR_YUV2BGR)

        
        cv2.namedWindow('imgView', cv2.WINDOW_NORMAL)
        cv2.imshow('imgView',sample_image)
        

        cv2.waitKey(0)
        cv2.destroyAllWindows()
        '''
         if cv2.getWindowProperty('imgView',cv2.WND_PROP_VISIBLE) <= 0:
            cv2.destroyAllWindows()
            exit(1)



        if not(cv2.cvGetWindowHandle('imgView')):
                    cv2.destroyAllWindows()
                    exit(1)



        '''



In [12]:
A()

무한 루프